## reference

### - ASR [whisper(OpenAI)](https://github.com/openai/whisper)
### - NLU [thkkvui/xlm-roberta-base-finetuned-JaQuAD(HuggingFace)](https://huggingface.co/thkkvui/xlm-roberta-base-finetuned-JaQuAD)
### - Record [pyaudio](https://people.csail.mit.edu/hubert/pyaudio/docs/#class-pyaudio)
### - TTS [TTS(coqui-ai)](https://github.com/coqui-ai/TTS)

In [89]:
# 必要なモジュールのインストール

!pip install -q openai-whisper
!pip install -q torch
!pip install -q transformers
!pip install -q datasets

In [1]:
import torch

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='mps')

## ASR

In [3]:
import whisper
asr_model = whisper.load_model("base")

## NLU

## load model (from huggingface hub)

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model_name = "thkkvui/xlm-roberta-base-finetuned-JaQuAD"
nlu_model = (AutoModelForQuestionAnswering.from_pretrained(model_name).to(device))
tokenizer = AutoTokenizer.from_pretrained(model_name)

## load model (from local)

In [4]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

# load model
nlu_model = (AutoModelForQuestionAnswering.from_pretrained("./output").to(device))
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

## 性能確認

In [5]:
text = """
私は音声アシスタントです。\n
この7月で5歳になりました。\n
今札幌に住んでいます。\n
昨日は帯広に出かけました。\n
好きなイベントはバルーンフェスティバルです。\n
好きな食べ物はバタークッキーです。\n
趣味はカヌーです。
"""

questions = ["昨日はどこへ出かけましたか？", 
             "あなたの名前は何ですか？", 
             "何歳ですか？",
             "あなたの趣味を教えてください。", 
             "あなたが好きなイベントは何ですか？"]

for question in questions:
    
    inputs = tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors="pt").to(device)
    
    with torch.no_grad():
        output = nlu_model(**inputs)

    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)

    answer_tokens = inputs.input_ids[0, answer_start : answer_end + 1]
    answer = tokenizer.decode(answer_tokens)

    print(f"質問: {question} -> 回答: {answer}")

質問: 昨日はどこへ出かけましたか？ -> 回答: 帯広
質問: あなたの名前は何ですか？ -> 回答: 音声アシスタント
質問: 何歳ですか？ -> 回答: 5歳
質問: あなたの趣味を教えてください。 -> 回答: カヌー
質問: あなたが好きなイベントは何ですか？ -> 回答: バルーンフェスティバル


## Record

In [ ]:
# 必要なモジュールのインストール

!pip install -q pyaudio

In [14]:
# questions

# "昨日はどこへ出かけましたか？"
# "あなたの名前は何ですか？"
# "何歳ですか？"
# "あなたの趣味を教えてください。"
# "あなたが好きなイベントは何ですか？"

In [14]:
import pyaudio
import wave

record_time = 8
record_filepath = "record.wav"

FORMAT = pyaudio.paInt16        
rate = 44100
chunk = 2**11        
audio = pyaudio.PyAudio()

stream = audio.open(format=FORMAT,
                    input=True,
                    rate=rate, 
                    frames_per_buffer=chunk,
                    channels=1,
)

print(f"Speak to your microphone for {record_time} sec...")
frames = []
for i in range(0, int(rate / chunk * record_time)):
    data = stream.read(chunk)
    frames.append(data) 
print ("Great!")

stream.stop_stream()
stream.close()
audio.terminate()

wf = wave.open(record_filepath, 'wb')
wf.setnchannels(1)
wf.setsampwidth(audio.get_sample_size(FORMAT))
wf.setframerate(rate)
wf.writeframes(b''.join(frames))
wf.close()

Speak to your microphone for 8 sec...
Great!


### data preprocessing

In [8]:
# 必要なモジュールのインストール

!pip install -q mecab-python3
!pip install -q alkana

In [16]:
asr_text = asr_model.transcribe(record_filepath, verbose=False, language="ja")
print(f'{asr_text["text"]}')

In [17]:
import re
import MeCab
import alkana
import pandas as pd

# トークン化
al_re = re.compile(r'^[a-zA-Z]+$')
def is_al(text):
    return al_re.match(text) is not None

tmp_text = asr_text["text"] #"helloテレビを見ました"
wakati = MeCab.Tagger('-Owakati')
wakati_output = wakati.parse(tmp_text)
print(wakati_output)


# 英語検索
df = pd.DataFrame(wakati_output.split(" "),columns=["word"])
df["en_word"] = df["word"].apply(is_al)
df["katakana"] = df["word"].apply(alkana.get_kana)
print(df)
print(" ")
# カタカナ変換
df = df[df["en_word"] == True]
dict_rep = dict(zip(df["word"], df["katakana"]))

if len(df) > 0:
    for word, katakana in dict_rep.items():
        asr_text = tmp_text.replace(word, katakana)
else:
    asr_text = tmp_text
    
print(asr_text)

昨日 は どこ へ 出かけ まし た か 

  word  en_word katakana
0   昨日    False     None
1    は    False     None
2   どこ    False     None
3    へ    False     None
4  出かけ    False     None
5   まし    False     None
6    た    False     None
7    か    False     None
8   \n    False     None
 
昨日はどこへ出かけましたか


## TTS

In [27]:
# 必要なモジュールのインストール

!pip install -q TTS

In [26]:
from TTS.api import TTS

# Japanese
TTS.list_models()[38] 

In [25]:
# Download model
model_name = TTS.list_models()[38]
tts = TTS(model_name)

In [21]:
# output

inputs = tokenizer.encode_plus(asr_text, text, add_special_tokens=True, return_tensors="pt").to(device)

with torch.no_grad():
    output = nlu_model(**inputs)

answer_start = torch.argmax(output.start_logits)
answer_end = torch.argmax(output.end_logits)

answer_tokens = inputs.input_ids[0, answer_start : answer_end + 1]
answer_text = tokenizer.decode(answer_tokens)

In [22]:
tts_filepath = "output.wav"
tts.tts_to_file(answer_text, speaker=tts.speakers, language=tts.languages, file_path=tts_filepath)

 > Text splitted to sentences.
['帯広']
 > Processing time: 0.13511395454406738
 > Real-time factor: 0.10114281293103904


'output.wav'

In [24]:
import librosa
import IPython

def sound():
    y, sr = librosa.load(f"{tts_filepath}")
    return IPython.display.Audio(data = y, rate=sr, autoplay=True)

sound()